In [ ]:
import numpy as np

In [ ]:
def Differentially_Expressed_Genes_Constructor(dataframe_strict, asc_param):

  diff_exp_genes = {}

  timePoint_cluster_numbers = dataframe_strict['group'].unique().tolist()

  number_of_clusters =  len(timePoint_cluster_numbers)

  for cluster_number in timePoint_cluster_numbers:

    diff_dataFrame_strict_sorted = dataframe_strict.sort_values(['group', 'scores'], ascending=[True, asc_param] )
    genes_list = diff_dataFrame_strict_sorted.loc[diff_dataFrame_strict_sorted['group'] == cluster_number, 'names'].tolist()
    diff_exp_genes[cluster_number] = genes_list

  return diff_exp_genes, timePoint_cluster_numbers, number_of_clusters

In [ ]:
# Standard Deviation Ranked Genes
def Standard_Deviation_Genes_Constructor(cluster_gene_average_tuple_list, gene):

  gene_standard_deviation_tuple = tuple()

  take_SD = 0
  gene_average_list = []

  for mean_exp_tuple_vals in cluster_gene_average_tuple_list:
    if mean_exp_tuple_vals[2] > 0:
      take_SD += 1
    gene_average_list.append(mean_exp_tuple_vals[2])

  if take_SD >= 2:

    standard_deviation = np.std(gene_average_list)
    gene_standard_deviation_tuple = (gene, standard_deviation)

  return gene_standard_deviation_tuple

In [ ]:
#UNIQUE GENES

def Unique_SD_Genes_Constructor(timepoint_names, diff_dataFrame_timePoints_strict, timepoints_anndata):

  all_timepoint_unique_genes_dict = {}
  all_timepoint_standard_deviation_genes_dict = {}
  asc_param = False

  for timepoint_name, dataframe_strict, anndata in zip(timepoint_names, diff_dataFrame_timePoints_strict, timepoints_anndata):

    print('Average Runtime 28 minutes'+'\n')

    diff_exp_genes_strict_cutoff = {}
    diff_exp_sig_genes, timePoint_cluster_numbers, number_of_clusters = Differentially_Expressed_Genes_Constructor(dataframe_strict, asc_param)

    timepoint_unique_genes = {}
    timepoint_absolute_spread_genes = {}
    timepoint_standard_deviation_genes_list = []

    all_timepoint_genes_set = set(dataframe_strict['names'].tolist())

    length = len(all_timepoint_genes_set)

    for x, gene in enumerate(all_timepoint_genes_set):

      print('\rWorking on      Timepoint: '+timepoint_name+':     Gene: ' + gene+'  '+ str(x + 1) + ':' + str(length), end='')

      cluster_gene_average_tuple_list = []
      total_means = []

      for cluster_number in timePoint_cluster_numbers:
        ind_means = float((anndata.raw[anndata.obs['E_I_M'].isin([cluster_number]),:][:,gene].X > 0).mean(0))
        total_means.append(ind_means)
      neuronal_gene_mean = sum(total_means)/len(total_means)

      for cluster_number in timePoint_cluster_numbers:

        cluster_gene_mean = float((anndata.raw[anndata.obs['E_I_M'].isin([cluster_number]),:][:,gene].X > 0).mean(0)[0][0])
        gene_cluster_gene_mean_tuple = (cluster_number,gene,cluster_gene_mean)
        cluster_gene_average_tuple_list.append(gene_cluster_gene_mean_tuple)

        if gene in diff_exp_sig_genes[cluster_number]:
          neuronal_gene_mean_excluding_clust = ((neuronal_gene_mean*number_of_clusters)-(cluster_gene_mean)) / (number_of_clusters - 1)

          if neuronal_gene_mean_excluding_clust == 0:
            mean_expression_ranking = float('inf')

          elif neuronal_gene_mean_excluding_clust > 0:
            mean_expression_ranking = cluster_gene_mean/neuronal_gene_mean_excluding_clust
          gene_mean_expression_tuple = (gene,mean_expression_ranking)

          if cluster_number in timepoint_unique_genes:
            list_of_mean_ratio_ranking = timepoint_unique_genes.get(cluster_number)
            list_of_mean_ratio_ranking.append(gene_mean_expression_tuple)
            timepoint_unique_genes[cluster_number] = list_of_mean_ratio_ranking
          elif cluster_number not in timepoint_unique_genes:
            timepoint_unique_genes[cluster_number] = [gene_mean_expression_tuple]


      sd_tup = Standard_Deviation_Genes_Constructor(cluster_gene_average_tuple_list, gene)
      if sd_tup:
        timepoint_standard_deviation_genes_list.append(sd_tup)
      else:
        pass

    timepoint_unique_genes_sorted = {}

    for cluster_number in timepoint_unique_genes:
      unique_genes_list = timepoint_unique_genes.get(cluster_number)
      timepoint_unique_genes_sorted[cluster_number] = sorted(unique_genes_list, key=lambda t: t[1], reverse = True)

    timepoint_standard_deviation_genes_sorted = sorted(timepoint_standard_deviation_genes_list, key=lambda t: t[1], reverse = True)

    all_timepoint_unique_genes_dict[timepoint_name] = timepoint_unique_genes_sorted
    all_timepoint_standard_deviation_genes_dict[timepoint_name] = timepoint_standard_deviation_genes_sorted
    print('\n')

  return all_timepoint_unique_genes_dict, all_timepoint_standard_deviation_genes_dict